In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score



In [2]:
df = pd.read_csv("spending_l9_dataset.csv")

# Handle missing values via median
df["Income_$"] = df["Income_$"].fillna(df["Income_$"].median())
df["SpendingScore"] = df["SpendingScore"].fillna(df["SpendingScore"].median())

# Select features
X = df[["Income_$", "SpendingScore"]]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Dataset Loaded and Scaled. First 5 rows of scaled data:")
print(X_scaled[:5])

Dataset Loaded and Scaled. First 5 rows of scaled data:
[[-0.62783049  0.72998073]
 [-0.89031514  1.07359091]
 [-0.92312573  1.11176982]
 [-0.89031514  0.53908619]
 [-0.95593631  1.11176982]]


In [3]:
print("=== ELBOW CHECK (SSE) ===")
sse_list = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    kmeans.fit(X_scaled)
    sse_list.append(kmeans.inertia_)
    print(f"k={k} → SSE={kmeans.inertia_:.2f}")

=== ELBOW CHECK (SSE) ===


NameError: name 'RANDOM_STATE' is not defined

In [ ]:

# 3) Model Training (Pick K)

CHOSEN_K = 3 # Change this based on your SSE results
final_model = KMeans(n_clusters=CHOSEN_K, random_state=RANDOM_STATE, n_init=10)
df['Cluster'] = final_model.fit_predict(X_scaled)


# 4) Evaluate Clustering

sil_score = silhouette_score(X_scaled, df['Cluster'])
dbi_score = davies_bouldin_score(X_scaled, df['Cluster'])

print(f"\nChosen K: {CHOSEN_K}")
print(f"Silhouette Score : {sil_score:.3f}")
print(f"Davies–Bouldin   : {dbi_score:.3f}")


Chosen K: 3
Silhouette Score : 0.611
Davies–Bouldin   : 0.584


C:\Users\hp\anaconda1\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [ ]:

# 5) Cluster Centers (Original Units)

centers_scaled = final_model.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled)

centers_df = pd.DataFrame(
    centers_original, 
    columns=["Income_$", "SpendingScore"]
).round(2)
centers_df.index.name = "Cluster"

print("\n=== CLUSTER CENTERS (Original Units) ===")
print(centers_df)


# 6) Sanity Check & Save

print("\n=== SANITY CHECK (3 Samples) ===")
print(df[["Income_$", "SpendingScore", "Cluster"]].sample(3))

# Save output
df.to_csv("spending_labeled_clusters.csv", index=False)
print("\nLabeled dataset saved as: spending_labeled_clusters.csv")


=== CLUSTER CENTERS (Original Units) ===
         Income_$  SpendingScore
Cluster                         
0           37.81          71.19
1           32.43          22.45
2           97.75          78.08

=== SANITY CHECK (3 Samples) ===
     Income_$  SpendingScore  Cluster
142        25             13        1
22         24             90        0
19         18             98        0

Labeled dataset saved as: spending_labeled_clusters.csv
